In [ ]:
### Import the packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv("crypto.csv")
df = df.set_index('Date')
dff = df.drop(["DOGE"], axis = 1)  ## dropping the DogeCoin from dataset


In [ ]:
### Computing returns
# It is common practice in portfolio optimization to take log of returns for calculations of covariance and 
# correlation.

returns = np.log(dff/dff.shift(1)) 
returns = returns.dropna()

er = returns.describe().loc['mean']  ## expected returns 
std = returns.describe().loc['std'] ## standard deviations
var = std**2  ## variance 
cov_matrix = returns.cov() ## covariance matrix
sns.heatmap(returns.corr(),annot=True, cmap = 'Blues') ## heatmap for correlation matrix 




In [ ]:
### Plots 

dff.plot(figsize = (14,7), fontsize=11) ## plotting the prices of assets over time
plt.ylabel('Price')
plt.show()


returns.plot(title = '',figsize = (14,7), fontsize=11)  ## plotting daily returns
plt.ylabel('Daily Returns')
plt.show()

In [ ]:
### Plotting distribution of returns 

from scipy.stats import norm
mu, std = norm.fit(returns) 

# Plot the histogram.
plt.hist(returns)
 
    
# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
  
plt.plot(x, p, 'k', linewidth=2)
title = "Fit Values: {:.2f} and {:.2f}".format(mu, std)
plt.title(title)
  
plt.show()


In [ ]:
### Data Analysis ###

risk_free_rate = 0.0002

# empty lists to store returns, volatility and weights of imiginary portfolios
port_returns = []
port_volatility = []
crypto_weights = []
sharpe_ratio = []

# set the number of combinations for imaginary portfolios
num_assets = len(returns.columns)
num_portfolios = 10000

# populate the empty lists with each portfolios returns,risk and weights
for single_portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    returns1 = np.dot(weights, er) * 365
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))* np.sqrt(365)
    port_returns.append(returns1)
    port_volatility.append(volatility)
    crypto_weights.append(weights)
    
    sharpe = (returns1 - risk_free_rate)/volatility
    sharpe_ratio.append(sharpe)

# a dictionary for Returns and Risk values of each portfolio
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

# extend original dictionary to accomodate each ticker and weight in the portfolio
for counter,symbol in enumerate(returns.columns):
    portfolio[symbol+' weight'] = [weight[counter] for weight in crypto_weights]
    
df = pd.DataFrame(portfolio)
df.head()

In [ ]:
df['Sharpe Ratio'].argmax()

In [ ]:
df.iloc[df['Sharpe Ratio'].argmax()]['Volatility']

In [ ]:
df.iloc[df['Volatility'].argmax()]['Volatility']

In [ ]:
df.iloc[df['Volatility'].argmin()]['Volatility']

In [ ]:
### Plotting the Efficient Frontier 
plt.style.use('seaborn')
df.plot.scatter(x='Volatility', y='Returns', figsize=(10, 8), grid=True)
plt.scatter(x=df.iloc[df['Sharpe Ratio'].argmax()]['Volatility'], y=df.iloc[df['Sharpe Ratio'].argmax()]['Returns'], marker='*', color = 'r',s=150, label='Maximum Sharpe ratio')
plt.scatter(x=df.iloc[df['Volatility'].argmin()]['Volatility'], y=df.iloc[df['Volatility'].argmax()]['Returns'], marker='o', color = 'g',s=150, label='Minimum Volatilty')



plt.legend()
plt.xlabel('Volatility/Standard Deviation')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
print("-"*80)
print("Maximum Sharpe Ratio Portfolio Allocation\n")
print("Annualised Return:", df.iloc[df['Sharpe Ratio'].argmax()]['Returns'])
print("Annualised Volatility:", df.iloc[df['Sharpe Ratio'].argmax()]['Volatility'])
print("\n")
print("Allocation: \n\n",df.iloc[df['Sharpe Ratio'].argmax()][3:])

print("-"*80)
print("Minimum Volatility Portfolio Allocation\n")
print("Annualised Return:", df.iloc[df['Volatility'].argmin()]['Returns'])
print("Annualised Volatility:", df.iloc[df['Volatility'].argmin()]['Volatility'])
print("\n")
print("Allocation: \n\n",df.iloc[df['Volatility'].argmin()][3:])